# Lab | Making predictions with logistic regression

In this lab, you will be using the [Sakila](https://dev.mysql.com/doc/sakila/en/) database of movie rentals.

In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.


### Instructions

1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.
2. Create a query to get the list of films and a boolean indicating if it was rented last month (May 2005). This would be our target variable.
3. Read the data into a Pandas dataframe.
4. Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.
5. Create a logistic regression model to predict this variable from the cleaned data.
6. Evaluate the results.


## Import libraries and get database password

In [3]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

········


## Get database data through sql

In [4]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = ''' select title, rental_duration, rental_rate, length, replacement_cost, rating, category, rented_may
            from(
            select film_id,
            case
            when month(rental_date) = 5 then True
            else False
            end as rented_may
            from rental
            join inventory using(inventory_id)
            join film using(film_id)
            join film_category using(film_id)
            join category using (category_id)
            where year(rental_date) = 2005 and month(rental_date) = 5
            group by film_id) t1
            right join(
            select rental_date, title, film_id, rental_duration, rental_rate, length, replacement_cost, rating, category.name as category
            from rental
            join inventory using(inventory_id)
            join film using(film_id)
            join film_category using(film_id)
            join category using (category_id)
            where year(rental_date) = 2005
            group by film_id
            ) t2
            using (film_id)
            order by title asc'''

data = pd.read_sql_query(query, engine)
data.head(30)


,title,rental_duration,rental_rate,length,replacement_cost,rating,category,rented_may
0,ACADEMY DINOSAUR,6,0.99,86,20.99,PG,Documentary,1.0
1,ACE GOLDFINGER,3,4.99,48,12.99,G,Horror,NaN
2,ADAPTATION HOLES,7,2.99,50,18.99,NC-17,Documentary,1.0
3,AFFAIR PREJUDICE,5,2.99,117,26.99,G,Horror,1.0
4,AFRICAN EGG,6,2.99,130,22.99,G,Family,1.0
5,AGENT TRUMAN,3,2.99,169,17.99,PG,Foreign,1.0
6,AIRPLANE SIERRA,6,4.99,62,28.99,PG-13,Comedy,NaN
7,AIRPORT POLLOCK,6,4.99,54,15.99,R,Horror,1.0
8,ALABAMA DEVIL,3,2.99,114,21.99,PG-13,Horror,NaN
9,ALADDIN CALENDAR,6,4.99,63,24.99,NC-17,Sports,NaN


In [5]:
# data = pd.read_csv('movie_db.csv')
# data
# wont run on my machine, importing csv from groupmate with correct data

In [6]:
data['rented_may'] = data['rented_may'].fillna(0)
# the nans here represent movie was not rented in may so we fill with 0 / false 

In [7]:
data

,title,rental_duration,rental_rate,length,replacement_cost,rating,category,rented_may
0,ACADEMY DINOSAUR,6,0.99,86,20.99,PG,Documentary,1.0
1,ACE GOLDFINGER,3,4.99,48,12.99,G,Horror,0.0
2,ADAPTATION HOLES,7,2.99,50,18.99,NC-17,Documentary,1.0
3,AFFAIR PREJUDICE,5,2.99,117,26.99,G,Horror,1.0
4,AFRICAN EGG,6,2.99,130,22.99,G,Family,1.0
...,...,...,...,...,...,...,...,...
953,YOUNG LANGUAGE,6,0.99,183,9.99,G,Documentary,0.0
954,YOUTH KICK,4,0.99,179,14.99,NC-17,Music,0.0
955,ZHIVAGO CORE,6,0.99,105,10.99,NC-17,Horror,1.0
956,ZOOLANDER FICTION,5,2.99,101,28.99,R,Children,1.0


In [8]:
# Has to be categorized --- probably gonna drop this one
data['replacement_cost'].value_counts()
# drop 

20.99    55
21.99    55
22.99    54
29.99    52
12.99    52
27.99    51
13.99    50
14.99    48
11.99    47
17.99    46
10.99    46
26.99    45
19.99    45
23.99    44
25.99    41
9.99     40
28.99    40
18.99    40
24.99    37
16.99    36
15.99    34
Name: replacement_cost, dtype: int64

In [9]:
# Has to be categorized
data['rental_duration'].value_counts()
# data['duration'] = data['duration'].astype('object') 

6    203
3    197
4    194
5    186
7    178
Name: rental_duration, dtype: int64

In [10]:
# Has to be categorized 
data['rental_rate'].value_counts()
# data['duration'] = data['duration'].astype('object') 

0.99    326
4.99    320
2.99    312
Name: rental_rate, dtype: int64

In [11]:
# bining the movies. 0-90,90-120,150-max
data['length'].value_counts()

85     17
179    13
84     13
112    12
122    11
       ..
94      3
96      2
55      2
66      2
95      2
Name: length, Length: 140, dtype: int64

In [12]:
data = data.drop(['replacement_cost'], axis=1)
data.head()

,title,rental_duration,rental_rate,length,rating,category,rented_may
0,ACADEMY DINOSAUR,6,0.99,86,PG,Documentary,1.0
1,ACE GOLDFINGER,3,4.99,48,G,Horror,0.0
2,ADAPTATION HOLES,7,2.99,50,NC-17,Documentary,1.0
3,AFFAIR PREJUDICE,5,2.99,117,G,Horror,1.0
4,AFRICAN EGG,6,2.99,130,G,Family,1.0


In [14]:
# data = data.drop(['Unnamed: 0'], axis=1)
# data

In [15]:
# columns to categorical 
data.dtypes

title               object
rental_duration      int64
rental_rate        float64
length               int64
rating              object
category            object
rented_may         float64
dtype: object

In [16]:
data['rental_duration'] = data['rental_duration'].astype(object) 
data['rental_rate'] = data['rental_rate'].astype(object) 


In [17]:
data.dtypes

title               object
rental_duration     object
rental_rate         object
length               int64
rating              object
category            object
rented_may         float64
dtype: object

In [18]:
# binnames = ["short", "normal", "long", "extended"]
# bins = pd.cut(data['length'],[0,90,120,150,int(data['length'].max())], labels=binnames) 
# data['length'].value_counts()

lbl = ['short', 'normal', 'long', 'extended']
data['length'] = pd.cut(data['length'],[0,90,120,150,int(data['length'].max())], labels=lbl)
data['length'].value_counts()

short       311
extended    233
normal      207
long        207
Name: length, dtype: int64

In [19]:
# X/Y split 
y = data['rented_may']
X = data.drop('rented_may', axis=1)

# train/test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# This encoder only will be run one time
onehotencoder = OneHotEncoder(drop='first', handle_unknown = 'ignore').fit(X_train)

# One hot encoder funtion
def encode_cats(X_cat):
    
    encoded = onehotencoder.transform(X_cat).toarray()
    cols = onehotencoder.get_feature_names(input_features=X_cat.columns)
    return pd.DataFrame(encoded, columns=cols)

X_train = encode_cats(X_train)
X_train.head()

/Users/lucaalfano/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,title_ACE GOLDFINGER,title_ADAPTATION HOLES,title_AFFAIR PREJUDICE,title_AFRICAN EGG,title_AGENT TRUMAN,title_AIRPLANE SIERRA,title_AIRPORT POLLOCK,title_ALABAMA DEVIL,title_ALADDIN CALENDAR,title_ALAMO VIDEOTAPE,...,category_Drama,category_Family,category_Foreign,category_Games,category_Horror,category_Music,category_New,category_Sci-Fi,category_Sports,category_Travel
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
X_test = encode_cats(X_test)
X_test.head()

/Users/lucaalfano/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:170: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/lucaalfano/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,title_ACE GOLDFINGER,title_ADAPTATION HOLES,title_AFFAIR PREJUDICE,title_AFRICAN EGG,title_AGENT TRUMAN,title_AIRPLANE SIERRA,title_AIRPORT POLLOCK,title_ALABAMA DEVIL,title_ALADDIN CALENDAR,title_ALAMO VIDEOTAPE,...,category_Drama,category_Family,category_Foreign,category_Games,category_Horror,category_Music,category_New,category_Sci-Fi,category_Sports,category_Travel
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [22]:
from sklearn import neighbors
clf =neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform')
clf.fit(X_train, y_train)
predictions_clf = clf.predict(X_test)
clf.score(X_test, y_test)

0.6458333333333334

In [23]:
pd.Series(y_test).value_counts()

1.0    136
0.0     56
Name: rented_may, dtype: int64

In [24]:
pd.Series(predictions_clf).value_counts()

1.0    172
0.0     20
dtype: int64

In [25]:
from sklearn.linear_model import LogisticRegression

classification = LogisticRegression(solver='saga').fit(X_train, y_train)
predictions = classification.predict(X_test)
classification.score(X_test, y_test)

0.7083333333333334

In [26]:
pd.Series(y_test).value_counts()

1.0    136
0.0     56
Name: rented_may, dtype: int64

In [27]:
pd.Series(predictions_clf).value_counts()

1.0    172
0.0     20
dtype: int64